# Temperature Control

Import necessary packages

In [1]:
from distutils.log import error
import board
import digitalio
import adafruit_max31856
import time
from datetime import datetime as dt
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np
import keyboard
from tclab import clock, setup, Historian, Plotter
import PID
import os
import pandas
from IPython.display import clear_output

Get current directory so we can save log files in a separate folder.

In [2]:
ROOT_DIR = os.path.realpath(os.path.join(os.path.dirname("Temperature Control.ipynb")))

Define the temperature and error logging functions that output to csv and text files respectivly.

In [3]:
def log_error(f_name,error,time,first):
    if first==True:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'w',encoding='UTF8',newline='') as err_log_file:
            err_log_file.write('')
    else:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'a',encoding='UTF8',newline='') as err_log_file:
            err_log_file.write("\n".join('{} at t= {}'.format(error,str(time))))

def log_temps(f_name,header,data,first):
    if first==True:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'w',encoding='UTF8', newline='') as temp_log_file:
            temp_log_w=csv.writer(temp_log_file)    #temperature log file
            temp_log_w.writerow(header)
    else:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'a',encoding='UTF8', newline='') as temp_log_file:
            temp_log_w=csv.writer(temp_log_file)    #temperature log file
            temp_log_w.writerow(data)

Create the temperature and error log files.

In [4]:
data_f_name=ROOT_DIR+'/Logs/Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
data_header=['Rt', 'temp_ch', 'temp_hex_f', 'temp_hex_b', 'temp_chamber', 'MV1','Heater17 On', 'MV2','Heater18 On']
log_temps(data_f_name,data_header,[0,0,0,0,0,0,0,0],True)
updated=False

Define the coldhead, heat exchanger, chamber, and heater entities that will be controled.

In [5]:
# Create sensor object, communicating over the board's default SPI bus
spi = board.SPI()

# allocate a CS pin and set the direction
cs13 = digitalio.DigitalInOut(board.D13)
cs13.direction = digitalio.Direction.OUTPUT
cs16 = digitalio.DigitalInOut(board.D16)
cs16.direction = digitalio.Direction.OUTPUT
cs25 = digitalio.DigitalInOut(board.D25)
cs25.direction = digitalio.Direction.OUTPUT
cs26 = digitalio.DigitalInOut(board.D26)
cs26.direction = digitalio.Direction.OUTPUT
HeaterF = digitalio.DigitalInOut(board.D17)
HeaterF.direction= digitalio.Direction.OUTPUT
HeaterB = digitalio.DigitalInOut(board.D18)
HeaterB.direction = digitalio.Direction.OUTPUT


# create a thermocouple object with the above
ColdHead = adafruit_max31856.MAX31856(spi, cs13,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
HeatExF = adafruit_max31856.MAX31856(spi, cs26,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
HeatExB = adafruit_max31856.MAX31856(spi, cs25,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
Chamber = adafruit_max31856.MAX31856(spi, cs16,thermocouple_type=adafruit_max31856.ThermocoupleType.T)


Initiate PID controls for Front Heat Exchanger

In [6]:
HeaterF.value = False
HeatF_on=False
targetT1 = -200   #initial inputs for PID control
P1 = 0.7*0.6
I1 = 0.7*1.2/58
D1 = 3*0.7*58/40

controllerF = PID.PID(P1, I1, D1)        # create pid control
controllerF.SetPoint = targetT1             # initialize
controllerF.setSampleTime(2)

Initiate PID controls for Back Heat Exchanger

Pretty good
P1 = 0.6
I1 = 1*1.2/65
D1 = 3*1*65/40

In [7]:
HeaterB.value = False 
HeatB_on = False
targetT2 = -200
P2 = 0.7*0.6
I2 = 0.7*1.2/58
D2 = 3*0.7*58/40

controllerB = PID.PID(P2, I2, D2)
controllerB.SetPoint = targetT2
controllerB.setSampleTime(2)

Read the coldhead, heat exhanger and chamber temperatures then log them.

In [8]:
temp_coldhead=ColdHead.temperature
temp_HeatExF=HeatExF.temperature
temp_HeatExB=HeatExB.temperature
temp_chamber=Chamber.temperature

log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, controllerF.output,"False", controllerB.output, "False"],False)

Create the temperature plot

In [9]:
plot_window = 1
coldhead_temps = np.array(np.zeros([plot_window]))
HeatExF_temps = np.array(np.zeros([plot_window]))
HeatExB_temps = np.array(np.zeros([plot_window]))
chamber_temps = np.array(np.zeros([plot_window]))
coldhead_temps[0] = temp_coldhead # Sets the first value of each line
HeatExF_temps[0] = temp_HeatExF
HeatExB_temps[0] = temp_HeatExB
chamber_temps[0] = temp_chamber
time_stamps = []
#np.array(np.zeros([plot_window]))
for i in range(plot_window):
    time_stamps.append(dt.now().strftime('%H:%M:%S'))
    #x_var.append(time.asctime(time.time()))
plt.ion()
fig, ax = plt.subplots(figsize=(12,5))
chline,  = ax.plot(time_stamps, coldhead_temps, label='Cold Head')
hexfline, = ax.plot(time_stamps, HeatExF_temps, label='Heat Exchanger Front')
hexbline, = ax.plot(time_stamps, HeatExB_temps, label='Heat Exchanger Back')
chamberline, =ax.plot(time_stamps, chamber_temps, label='Chamber')
ax.locator_params(tight=True, nbins=10)
ax.legend()
ax.set_xlabel('Time (Hour:Min)')
ax.set_ylabel('Temperature (Celsius)')

/tmp/ipykernel_2951/3583554292.py:21: UserWarning: 'set_params()' not defined for locator of type <class 'matplotlib.category.StrCategoryLocator'>
  ax.locator_params(tight=True, nbins=10)


Text(0, 0.5, 'Temperature (Celsius)')

Loop that reads the temperatures of the coldhead, heat exhanger and chamber logs them, updates the PID control and runs the heater, then plots the tmperatures.

In [ ]:
print_log=[]
print_head=["Time" ,"Coldhead", "Heat Exchanger Front", "Heat Exchanger Back", "Chamber", "PID1 Output", "Heater17 Status", "PID2 Output", "Heater18 Status"]
try:
    while True:
        now = time.time()
        clear_output(wait=True)
        if os.stat(data_f_name).st_size>= 4194304:
                data_f_name=ROOT_DIR+'/Logs/Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
                updated=True
        temp_coldhead=ColdHead.temperature
        temp_HeatExF=HeatExF.temperature
        temp_HeatExB=HeatExB.temperature
        temp_chamber=Chamber.temperature
        controllerF.update(temp_HeatExF) # compute manipulated variable
        controllerB.update(temp_HeatExB)
        MV1 = controllerF.output # apply
        MV2 = controllerB.output
        if MV1 > 0:
            HeaterF.value = True
            HeatF_on=True
        else:
            HeaterF.value = False
            HeatF_on=False
        if MV2 > 0:
            HeaterB.value = True
            HeatB_on = True
        else:
            HeaterB.value = False
            HeatB_on = False
        if HeatF_on:
            HeatF_status = 'On'
        else:
            HeatF_status = 'Off'
        if HeatB_on:
            HeatB_status = 'On'
        else:
            HeatB_status = 'Off'
        if updated:
            log_temps(data_f_name,data_header,[0,0,0,0,0,0,0,0,0],True)
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, HeatF_status , MV2, HeatB_status],False)
            updated = False
        else:
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, HeatF_status , MV2, HeatB_status],False)            
        if len(print_log)<=20:
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, HeatF_status , MV2, HeatB_status])
        else:
            print_log.pop(0)
            print_log.append([dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_coldhead, temp_HeatExF, temp_HeatExB, temp_chamber, MV1, HeatF_status, MV2, HeatB_status])
        to_print=pandas.DataFrame(print_log,index=None, columns=print_head)
        print(to_print.to_markdown())
        elapsed = time.time() - now  # how long was it running?
        #print(elapsed)
        coldhead_temps = np.append(coldhead_temps, temp_coldhead)
        HeatExF_temps = np.append(HeatExF_temps, temp_HeatExF)
        HeatExB_temps = np.append(HeatExB_temps, temp_HeatExB)
        chamber_temps = np.append(chamber_temps, temp_chamber)
        time_stamps.append(dt.now().strftime('%H:%M:%S'))
        #coldhead_temps = coldhead_temps[1: plot_window + 1]
        #HeatExF_temps = HeatExF_temps[1:plot_window+1]
        #HeatExB_temps = HeatExB_temps[1:plot_window+1]
        #chamber_temps = chamber_temps[1:plot_window+1]
        #time_stamps = time_stamps[1: plot_window + 1]
        if len(coldhead_temps)<=300:
            chline.set_ydata(coldhead_temps)
            chline.set_xdata(time_stamps)
            hexfline.set_ydata(HeatExF_temps)
            hexfline.set_xdata(time_stamps)
            hexbline.set_ydata(HeatExB_temps)
            hexbline.set_xdata(time_stamps)
            chamberline.set_ydata(chamber_temps)
            chamberline.set_xdata(time_stamps)
        else:
            chline.set_ydata(coldhead_temps[-300:])
            chline.set_xdata(time_stamps[-300:])
            hexfline.set_ydata(HeatExF_temps[-300:])
            hexfline.set_xdata(time_stamps[-300:])
            hexbline.set_ydata(HeatExB_temps[-300:])
            hexbline.set_xdata(time_stamps[-300:])
            chamberline.set_ydata(chamber_temps[-300:])
            chamberline.set_xdata(time_stamps[-300:])
        #ax.set_ylim(0,24)
        ax.set_xticks(time_stamps[-301::50])
        ax.set_xticklabels(time_stamps[-301::50],rotation=25)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()
        elapsed = time.time() - now  # how long was it running?
        #print(elapsed)
        try:time.sleep(2.-elapsed)
        except: time.sleep(1)

#Opens the relays (stops the heaters) when program interrupted
except Exception as e:
    HeaterF.value = False
    HeatF_on = False
   
    HeaterB.value = False
    HeatB_on = False
    print(e)
    print('Interrupted')

|    | Time                |   Coldhead |   Heat Exchanger Front |   Heat Exchanger Back |   Chamber |   PID1 Output | Heater17 Status   |   PID2 Output | Heater18 Status   |
|---:|:--------------------|-----------:|-----------------------:|----------------------:|----------:|--------------:|:------------------|--------------:|:------------------|
|  0 | 2022-10-13 15:37:21 |   -166.609 |               -141.82  |              -121.18  |  -15.1094 |      -24.7251 | Off               |      -33.3644 | Off               |
|  1 | 2022-10-13 15:37:23 |   -166.492 |               -141.641 |              -121.125 |  -15.1797 |      -24.7251 | Off               |      -33.3644 | Off               |
|  2 | 2022-10-13 15:37:25 |   -166.633 |               -141.844 |              -121.164 |  -15.0703 |      -24.6975 | Off               |      -33.4126 | Off               |
|  3 | 2022-10-13 15:37:27 |   -166.586 |               -141.859 |              -121.18  |  -15.0859 |      -24.6849 | Off   